In [1]:
import pandas as pd

#  Reading the CSV files
df = pd.read_csv("/content/census_data_2011_cleaned.csv")

In [3]:
df.head()

State              state       Age  Total Persons
0      1   Jammu & Kashmir   All ages       12541302
1      1   Jammu & Kashmir          0         275312
2      1   Jammu & Kashmir          1         274393
3      1   Jammu & Kashmir          2         285931
4      1   Jammu & Kashmir          3         286440

This code processes census data to aggregate population counts by state and specific age groups. Here's a breakdown of the logic:

1.  **Extracting 'All Ages' Total**: The first step isolates the total population for each state by filtering for 'Age' as 'All ages', then groups by 'state' and sums 'Total Persons' to create the `All_Ages_Total` column in `df_all_ages`. This gives us the overall population for each state.

2.  **Preparing Numeric Age Data**: Next, the original DataFrame is filtered to exclude 'All ages' rows, creating `df_filtered`. The 'Age' column is then converted to a numeric type, with any non-numeric values being coerced to `NaN`. This prepares the data for age grouping.

3.  **Categorizing Age Groups**: Using `numpy` and `pandas.cut`, the numeric 'Age' column in `df_filtered` is categorized into three distinct age groups: '0-5', '6-17', and '18-100'. Any `NaN` values (representing ages that were not stated or uninterpretable) are explicitly assigned to an 'Age Not Stated' group. This creates a new 'Age_Group' column.

4.  **Aggregating Age Group Totals**: The `df_filtered` DataFrame is then grouped by 'state' and 'Age_Group', and the 'Total Persons' are summed. The `unstack()` method pivots the 'Age_Group' categories into separate columns, making it easy to see the population for each age group per state.

5.  **Merging and Finalizing**: Finally, the `df_all_ages` (containing the overall state totals) and the aggregated age group DataFrame are merged based on the 'state' column. The age group columns are then renamed for clarity (e.g., 'Age_0_5', 'Age_18_100', 'Age_Not_Stated'), and the 'state' column is set as the DataFrame's index. The result is `df_final_census`, a comprehensive DataFrame showing the total population and detailed age group populations for each state.

In [ ]:
df_all_ages = df[df['Age'] == 'All ages'].groupby('state')['Total Persons'].sum().reset_index()
df_all_ages.rename(columns={'Total Persons': 'All_Ages_Total'}, inplace=True)
print("The DataFrame 'df_all_ages' has been successfully created:")
print(df_all_ages.head())

In [ ]:
df_filtered = df[df['Age'] != 'All ages'].copy()
df_filtered['Age'] = pd.to_numeric(df_filtered['Age'], errors='coerce')
print("DataFrame 'df_filtered' created with 'All ages' removed and 'Age' column converted to numeric:")
print(df_filtered.head())

In [ ]:
import numpy as np

bins = [-1, 5, 17, np.inf] # Define bins for age groups: 0-5, 6-17, 18-inf (for 18-100+)
labels = ['0-5', '6-17', '18-100'] # Labels for the age groups (should be len(bins) - 1)

df_filtered['Age_Group'] = pd.cut(df_filtered['Age'], bins=bins, labels=labels, right=True)

# Handle the NaN values which correspond to 'Age Not Stated'
df_filtered['Age_Group'] = np.where(df_filtered['Age'].isna(), 'Age Not Stated', df_filtered['Age_Group'])

print("DataFrame 'df_filtered' with 'Age_Group' column:")
print(df_filtered.head())

In [ ]:
df_final_census = pd.merge(df_all_ages, df_age_groups, on='state', how='left')
print("DataFrame 'df_final_census' created after merging 'All_Ages_Total' and age group data:")
print(df_final_census.head())

In [ ]:
df_final_census.rename(columns={'0-5': 'Age_0_5', '6-17': 'Age_6_17', '18-100': 'Age_18_100', 'Age Not Stated': 'Age_Not_Stated'}, inplace=True)
df_final_census.set_index('state', inplace=True)
print("Final DataFrame 'df_final_census' with renamed columns and 'state' as index:")
print(df_final_census.head())

In [14]:
df_final_census.head(35)

All_Ages_Total   Age_0_5  Age_18_100  Age_6_17  \
state                                                                        
Andaman & Nicobar Islands           380581     34721      267802     77588   
Andhra Pradesh                    84580777   7725898    57295958  18759505   
Arunachal Pradesh                  1383727    177268      792169    412288   
Assam                             31205576   3937549    19097809   8140172   
Bihar                            104099452  15837132    56154242  31665933   
Chandigarh                         1055450    101018      730731    223102   
Chhattisgarh                      25545198   3104853    15766641   6643584   
Dadra & Nagar Haveli                343709     43988      218060     81220   
Daman & Diu                         243247     23148      175912     43774   
Delhi                             16787941   1704152    11230123   3833026   
Goa                                1458545    123331     1073960    258766   
Gujarat                           60439692   6645698    39257179  14270870   
Haryana                           25351462   2882509    16189689   6237200   
Himachal Pradesh                   6864602    662663     4697162   1491095   
Jammu & Kashmir                   12541302   1718020     7520339   3278342   
Jharkhand                         32988134   4512458    19026375   9323703   
Karnataka                         61095297   6134041    41678490  13214158   
Kerala                            33406061   2965778    23960986   6439380   
Lakshadweep                          64473      6187       44443     13714   
Madhya Pradesh                    72626809   9142292    43751857  19617626   
Maharashtra                      112374333  11372408    75797230  24739049   
Manipur                            2855794    315761     1813472    717768   
Meghalaya                          2966889    487355     1580324    892803   
Mizoram                            1097206    145088      674161    277206   
Nagaland                           1978502    242896     1156375    577412   
Odisha                            41974218   4481140    27425652   9931966   
Pondicherry                        1247953    112992      888379    245309   
Punjab                            27743338   2612299    18943014   6127462   
Rajasthan                         68548437   9008858    40121438  19127920   
Sikkim                              610577     52896      403474    152936   
Tamil Nadu                        72147030   6358182    51405820  14303569   
Tripura                            3673917    391786     2442426    836221   
Uttar Pradesh                    199812341  25640761   112655896  59694024   
Uttarakhand                       10086292   1142931     6263404   2657179   
West Bengal                       91276115   9012951    61105075  20986452   

                            Age_Not_Stated  
state                                       
Andaman & Nicobar Islands              470  
Andhra Pradesh                      799416  
Arunachal Pradesh                     2002  
Assam                                30046  
Bihar                               442145  
Chandigarh                             599  
Chhattisgarh                         30120  
Dadra & Nagar Haveli                   441  
Daman & Diu                            413  
Delhi                                20640  
Goa                                   2488  
Gujarat                             265945  
Haryana                              42064  
Himachal Pradesh                     13682  
Jammu & Kashmir                      24601  
Jharkhand                           125598  
Karnataka                            68608  
Kerala                               39917  
Lakshadweep                            129  
Madhya Pradesh                      115034  
Maharashtra                         465646  
Manipur                               8793  
Meghalaya                             6407  
Mizoram         

In [15]:
#csv of aggregated result

In [ ]:
state,All_Ages_Total,Age_0_5,Age_18_100,Age_6_17,Age_Not_Stated
 Andaman & Nicobar Islands ,380581,34721,267802,77588,470
 Andhra Pradesh ,84580777,7725898,57295958,18759505,799416
 Arunachal Pradesh ,1383727,177268,792169,412288,2002
 Assam ,31205576,3937549,19097809,8140172,30046
 Bihar ,104099452,15837132,56154242,31665933,442145
 Chandigarh ,1055450,101018,730731,223102,599
 Chhattisgarh ,25545198,3104853,15766641,6643584,30120
 Dadra & Nagar Haveli ,343709,43988,218060,81220,441
 Daman & Diu ,243247,23148,175912,43774,413
 Delhi ,16787941,1704152,11230123,3833026,20640
 Goa ,1458545,123331,1073960,258766,2488
 Gujarat ,60439692,6645698,39257179,14270870,265945
 Haryana ,25351462,2882509,16189689,6237200,42064
 Himachal Pradesh ,6864602,662663,4697162,1491095,13682
 Jammu & Kashmir ,12541302,1718020,7520339,3278342,24601
 Jharkhand ,32988134,4512458,19026375,9323703,125598
 Karnataka ,61095297,6134041,41678490,13214158,68608
 Kerala ,33406061,2965778,23960986,6439380,39917
 Lakshadweep ,64473,6187,44443,13714,129
 Madhya Pradesh ,72626809,9142292,43751857,19617626,115034
 Maharashtra ,112374333,11372408,75797230,24739049,465646
 Manipur ,2855794,315761,1813472,717768,8793
 Meghalaya ,2966889,487355,1580324,892803,6407
 Mizoram ,1097206,145088,674161,277206,751
 Nagaland ,1978502,242896,1156375,577412,1819
 Odisha ,41974218,4481140,27425652,9931966,135460
 Pondicherry ,1247953,112992,888379,245309,1273
 Punjab ,27743338,2612299,18943014,6127462,60563
 Rajasthan ,68548437,9008858,40121438,19127920,290221
 Sikkim ,610577,52896,403474,152936,1271
 Tamil Nadu ,72147030,6358182,51405820,14303569,79459
 Tripura ,3673917,391786,2442426,836221,3484
 Uttar Pradesh ,199812341,25640761,112655896,59694024,1821660
 Uttarakhand ,10086292,1142931,6263404,2657179,22778
 West Bengal ,91276115,9012951,61105075,20986452,171637a